# Introducción

Utilizo un modelo basado en NLP para construir una columna de puntajes que luego se utilizará para predecir el valor de Mercado teniendo en cuenta los medios

El dataset a utilizar es el de final-articles.csv con el nombre de: Football News Articles     
url de Kaggle: https://www.kaggle.com/datasets/hammadjavaid/football-news-articles/data
---



# Imports y dependencias


In [25]:
!pip install pandas numpy spacy scikit-learn textblob
!python -m spacy download en_core_web_sm
!pip install sentence-transformers


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\marti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.8 MB 656.4 kB/s eta 0:00:20
      --------------------------------------- 0.2/12.8 MB 1.4 MB/s eta 0:00:10
     --- ------------------------------------ 1.1/12.8 MB 6.9 MB/s eta 0:00:02
     ------ --------------------------------- 2.2/12.8 MB 10.7 MB/s eta 0:00:01
     ------- -------------------------------- 2.5/12.8 MB 11.3 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 19.5 MB/s eta 0:00:01
     ------------------- -------------------- 6.2/12.8 MB 20.0 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 23.2 MB/s eta 0:00:01
     ------------------------------ --------- 9.6/12.8 MB 23.7 MB/s eta 0:00:01
     -------------------------------- ------ 10.7/12.8 MB 34.4 MB/s eta 0:00:01
     ----------------------------------- --- 11.6/


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\marti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\marti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
from transformers import pipeline
import spacy
import re
from textblob import TextBlob
from collections import defaultdict

In [27]:
df = pd.read_csv("final-articles.csv")

In [28]:
df

,link,author,title,content,publish-time,source
0,https://www.goal.com/en/news/explained-how-che...,Nizaar Kinsella,Explained: How Chelsea are affected by Abramov...,Roman Abramovich has been sanctioned by the Un...,Publish Time Not Found,Goal.com
1,https://www.goal.com/en/news/adidas-limited-ed...,Nick Khairi,Would you wear these in Sunday League? Adidas ...,"As ever, Adidas have been brave and bold in th...",Publish Time Not Found,Goal.com
2,https://www.goal.com/en/news/chelsea-owner-abr...,Andrew Steel,Chelsea owner Abramovich 'terrified of sanctio...,"Roman Abramovich is ""terrified"" of facing sanc...",Publish Time Not Found,Goal.com
3,https://www.goal.com/en/news/weekly-team-picks...,GOAL,Everything we've got on our radar this week: T...,"Here at GOAL, our team of writers, editors and...",Publish Time Not Found,Goal.com
4,https://www.goal.com/en/news/mwepu-prays-for-i...,Dennis Mabuka,Mwepu prays for injury-free season with Bright...,Zambia international Enock Mwepu is hoping to ...,Publish Time Not Found,Goal.com
...,...,...,...,...,...,...
11958,https://theanalyst.com/eu/2021/07/mls-western-...,Brady Olson,"MLS Western Conference Roundup: The Good, the ...","Last week, we took a look at the Eastern Confe...",30/07/2021 10:52,the-analyst
11959,https://theanalyst.com/eu/2021/07/the-analyst-...,Graham Bell,The Analyst 50: The Video Show,After a summer of frantic international footba...,29/07/2021 6:56,the-analyst
11960,https://theanalyst.com/eu/2021/07/scottish-pre...,Michael Reid,Rangers and Celtic Prepare For Battle: Scottis...,"The Scottish Premiership returns this weekend,...",29/07/2021 6:28,the-analyst
11961,https://theanalyst.com/eu/2021/07/players-to-w...,NaN,Players to Watch in 2021-22: The Analyst 50 (P...,After a summer of frantic international footba...,NaN,NaN


Elimino filas que tengan el mismo título y autor


In [29]:
df = df.drop_duplicates(subset=['title', 'author'])

Elimino la columna de links porque no es util para el modelo ni la de autor

In [30]:
df = df.drop(columns=['link'])
df = df.drop(columns=['author'])


Decido eliminar la columna source (se puede utilizar para calcular la calidad y profesionalidad de la noticia, pero es un analisis que aumenta mucho la complejidad)

In [31]:
df = df.drop(columns=['source'])

In [32]:
df

,title,content,publish-time
0,Explained: How Chelsea are affected by Abramov...,Roman Abramovich has been sanctioned by the Un...,Publish Time Not Found
1,Would you wear these in Sunday League? Adidas ...,"As ever, Adidas have been brave and bold in th...",Publish Time Not Found
2,Chelsea owner Abramovich 'terrified of sanctio...,"Roman Abramovich is ""terrified"" of facing sanc...",Publish Time Not Found
3,Everything we've got on our radar this week: T...,"Here at GOAL, our team of writers, editors and...",Publish Time Not Found
4,Mwepu prays for injury-free season with Bright...,Zambia international Enock Mwepu is hoping to ...,Publish Time Not Found
...,...,...,...
11958,"MLS Western Conference Roundup: The Good, the ...","Last week, we took a look at the Eastern Confe...",30/07/2021 10:52
11959,The Analyst 50: The Video Show,After a summer of frantic international footba...,29/07/2021 6:56
11960,Rangers and Celtic Prepare For Battle: Scottis...,"The Scottish Premiership returns this weekend,...",29/07/2021 6:28
11961,Players to Watch in 2021-22: The Analyst 50 (P...,After a summer of frantic international footba...,NaN


In [33]:
print(df.isnull().sum())

title            1
content          1
publish-time    10
dtype: int64


Elimino filas sin contenido ni titulo

In [34]:
df = df.dropna(subset=['content'])

In [35]:
df = df.dropna(subset=['title'])

Decido a reemplazar el valor las filas que no tienen fecha de publicación por la media de las fechas de publicación para no desperdiciar esas noticias

In [36]:
df['publish-time'] = df['publish-time'].replace('Publish Time Not Found', np.nan)
try:
    df['publish-time'] = pd.to_datetime(
        df['publish-time'],
        format='mixed',
        utc=True,
        errors='coerce'
    )
except Exception as e:
    print(f"Error en conversión: {e}")

valid_dates = df['publish-time'].dropna()
if not valid_dates.empty:
    timestamps = valid_dates.astype('int64')
    mean_timestamp = timestamps.mean()
    mean_date = pd.to_datetime(mean_timestamp, utc=True)
else:
    mean_date = pd.to_datetime('today', utc=True)

df['publish-time'] = df['publish-time'].fillna(mean_date)

print(f"- Fecha más temprana: {df['publish-time'].min()}")
print(f"- Fecha más reciente: {df['publish-time'].max()}")
print(f"- Fecha promedio usada: {mean_date}")
print(f"- Valores nulos restantes: {df['publish-time'].isna().sum()}")

- Fecha más temprana: 2021-01-09 12:25:00+00:00
- Fecha más reciente: 2024-12-02 11:28:00+00:00
- Fecha promedio usada: 2023-07-31 11:42:02.627859456+00:00
- Valores nulos restantes: 0


A partir de las fechas obtenidas calculo la antiguedad de las noticias y doy más peso a las noticias recientes utilizando un puntaje exponencial

In [37]:
current_date = pd.to_datetime('now', utc=True)
df['days_old'] = (current_date - df['publish-time']).dt.days
max_days = df['days_old'].max()
df['recency_score_exp'] = np.exp(-df['days_old'] / (max_days/3))
df

,title,content,publish-time,days_old,recency_score_exp
0,Explained: How Chelsea are affected by Abramov...,Roman Abramovich has been sanctioned by the Un...,2023-07-31 11:42:02.627859456+00:00,697,0.277254
1,Would you wear these in Sunday League? Adidas ...,"As ever, Adidas have been brave and bold in th...",2023-07-31 11:42:02.627859456+00:00,697,0.277254
2,Chelsea owner Abramovich 'terrified of sanctio...,"Roman Abramovich is ""terrified"" of facing sanc...",2023-07-31 11:42:02.627859456+00:00,697,0.277254
3,Everything we've got on our radar this week: T...,"Here at GOAL, our team of writers, editors and...",2023-07-31 11:42:02.627859456+00:00,697,0.277254
4,Mwepu prays for injury-free season with Bright...,Zambia international Enock Mwepu is hoping to ...,2023-07-31 11:42:02.627859456+00:00,697,0.277254
...,...,...,...,...,...
11958,"MLS Western Conference Roundup: The Good, the ...","Last week, we took a look at the Eastern Confe...",2021-07-30 10:52:00+00:00,1428,0.072207
11959,The Analyst 50: The Video Show,After a summer of frantic international footba...,2021-07-29 06:56:00+00:00,1429,0.072074
11960,Rangers and Celtic Prepare For Battle: Scottis...,"The Scottish Premiership returns this weekend,...",2021-07-29 06:28:00+00:00,1429,0.072074
11961,Players to Watch in 2021-22: The Analyst 50 (P...,After a summer of frantic international footba...,2023-07-31 11:42:02.627859456+00:00,697,0.277254


#Procesamiento de lenguaje natural

Instalo el modelo de lenguaje spacy en inglés ya que las noticias se encuentran en ese idioma

In [ ]:
try:
    nlp = spacy.load("en_core_web_sm")
except:
    !python -m spacy download en_core_web_sm
from sklearn.feature_extraction.text import TfidfVectorizer

def clean_text(text):
    text = re.sub(r'http\S+|@\w+|[^\w\s]', '', str(text))
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

df['clean_content'] = df['content'].apply(clean_text)

Realizo un checkpoint del dataset luego de aplicar la limpieza del texto 

In [ ]:
df.to_csv('checkpoint_clean_text.csv', index=False)

In [ ]:
df = pd.read_csv('checkpoint_clean_text.csv')

# Ingenierìa de Features

Creo columnas utiles sobre cada noticia , como qué jugadores y equipos están involucrados, si es un mensaje con sentimiento positivo o negativo.

In [ ]:
nlp = spacy.load("en_core_web_sm")

df_jugadores = pd.read_csv('players_ids.csv')

nombres_jugadores = {
    nombre.lower(): pid 
    for nombre, pid in zip(df_jugadores['Player'], df_jugadores['player_id'])
}

TOPIC_KEYWORDS = {
    'transfer': ['transfer', 'signing', 'bid', 'fichaje', 'traspaso'],
    'injury': ['injury', 'recovery', 'surgery', 'lesión', 'baja'],
    'contract': ['contract', 'extension', 'renewal', 'renovación'],
    'performance': ['goal', 'assist', 'hat-trick', 'gol', 'asistencia']
}

def safe_text_processing(text):
    if pd.isna(text):
        return ""
    return str(text).strip()

def encontrar_id_jugador(player_name):
    name_lower = player_name.lower()
    
    if name_lower in nombres_jugadores:
        return nombres_jugadores[name_lower]
    
    for nombre_db, pid in nombres_jugadores.items():
        if name_lower in nombre_db or nombre_db in name_lower:
            return pid
    
    return None

def extract_features_with_ids(text):
    try:
        text = safe_text_processing(text)
        if not text:
            return empty_features_with_ids()
        
        doc = nlp(text)
        
        players = list(set(ent.text for ent in doc.ents if ent.label_ == "PERSON"))
        
        player_ids = []
        for player in players:
            pid = encontrar_id_jugador(player)
            if pid is not None:
                player_ids.append(pid)
        
        sentiment = TextBlob(text).sentiment.polarity
        
        text_lower = text.lower()
        topics = {
            f"topic_{topic}": int(any(kw in text_lower for kw in keywords))
            for topic, keywords in TOPIC_KEYWORDS.items()
        }
        
        return {
            'sentiment': sentiment,
            'player_ids': list(set(player_ids)),
            **topics
        }
    
    except Exception as e:
        return empty_features_with_ids()

def empty_features_with_ids():
    return {
        'sentiment': 0,
        'player_ids': [],
        **{f"topic_{topic}": 0 for topic in TOPIC_KEYWORDS}
    }

features = df['clean_content'].apply(extract_features_with_ids)
features_df = pd.json_normalize(features)

Uno los resultados con el DataFrame original

In [ ]:
df = pd.concat([df, features_df], axis=1)

In [ ]:
df.to_csv('checkpoint_features.csv', index=False)

NameError: name 'df' is not defined

In [ ]:
df = pd.read_csv('checkpoint_features.csv')

In [ ]:
df = df.drop(columns=['days_old','content','title'])

Expando el dataset por cada player_id mencionado

In [ ]:
df["player_ids"] = df["player_ids"].str.replace(r'[\[\]]', '', regex=True)
df["player_ids"] = df["player_ids"].str.split(",")

def clean_and_convert_to_int(x):
    cleaned_ids = []
    for i in x:
        i = i.strip()
        if i.replace('.', '', 1).isdigit():
            cleaned_ids.append(int(float(i)))
    return cleaned_ids

df["player_ids"] = df["player_ids"].apply(clean_and_convert_to_int)

df = df.explode("player_ids")

df = df[df["player_ids"].notna()]

df

,publish-time,recency_score_exp,clean_content,sentiment,player_ids,topic_transfer,topic_injury,topic_contract,topic_performance
0,2023-07-31 11:42:02.627859456+00:00,0.277546,roman abramovich sanction united kingdom gover...,0.050376,55994,1,0,1,0
0,2023-07-31 11:42:02.627859456+00:00,0.277546,roman abramovich sanction united kingdom gover...,0.050376,105470,1,0,1,0
1,2023-07-31 11:42:02.627859456+00:00,0.277546,adida brave bold work launch new limited editi...,0.166458,581678,0,0,0,0
2,2023-07-31 11:42:02.627859456+00:00,0.277546,roman abramovich terrify face sanction british...,-0.018880,610881,0,0,0,0
2,2023-07-31 11:42:02.627859456+00:00,0.277546,roman abramovich terrify face sanction british...,-0.018880,105470,0,0,0,0
...,...,...,...,...,...,...,...,...,...
11935,2021-07-28 03:49:00+00:00,0.071958,harry kane oneseason wonder turn today seven c...,0.018599,418560,1,0,1,1
11935,2021-07-28 03:49:00+00:00,0.071958,harry kane oneseason wonder turn today seven c...,0.018599,132098,1,0,1,1
11935,2021-07-28 03:49:00+00:00,0.071958,harry kane oneseason wonder turn today seven c...,0.018599,355915,1,0,1,1
11935,2021-07-28 03:49:00+00:00,0.071958,harry kane oneseason wonder turn today seven c...,0.018599,234803,1,0,1,1


In [ ]:
mentions_table = df.groupby('player_ids').agg(
    total_mentions=('player_ids', 'count')  # Cuenta el número de filas (noticias) por jugador
).reset_index()

In [ ]:
mentions_table

,player_ids,total_mentions
0,3333,106
1,3455,2
2,4192,42
3,5023,1
4,7161,8
...,...,...
1643,1084667,6
1644,1176345,1
1645,1200289,1
1646,1272094,1


In [ ]:
sentiment_table = df.groupby("player_ids").agg(
    total_sentiment=("sentiment", "sum"),  # Suma acumulada
    avg_sentiment=("sentiment", "mean")    # Promedio
).reset_index()

In [ ]:
sentiment_table

,player_ids,total_sentiment,avg_sentiment
0,3333,12.221728,0.115299
1,3455,0.212278,0.106139
2,4192,4.799540,0.114275
3,5023,0.138602,0.138602
4,7161,1.019362,0.127420
...,...,...,...
1643,1084667,0.416138,0.069356
1644,1176345,0.062885,0.062885
1645,1200289,0.173349,0.173349
1646,1272094,0.092112,0.092112


In [ ]:
topics_table = df.groupby("player_ids").agg(
    total_transfer=("topic_transfer", "sum"),
    total_injury=("topic_injury", "sum"),
    total_contract=("topic_contract", "sum"),
    total_performance=("topic_performance", "sum")
).reset_index()

In [ ]:
topics_table

,player_ids,total_transfer,total_injury,total_contract,total_performance
0,3333,48,55,33,98
1,3455,0,1,1,2
2,4192,15,15,11,37
3,5023,0,0,0,1
4,7161,4,5,1,7
...,...,...,...,...,...
1643,1084667,3,4,2,6
1644,1176345,0,0,0,1
1645,1200289,0,0,0,1
1646,1272094,0,0,0,1


In [ ]:
total_recency_table = df.groupby("player_ids").agg(
    total_recency_score=("recency_score_exp", "mean")
).reset_index()

In [ ]:
total_recency_table

,player_ids,total_recency_score
0,3333,0.275923
1,3455,0.230630
2,4192,0.263786
3,5023,0.277546
4,7161,0.254314
...,...,...
1643,1084667,0.265712
1644,1176345,0.277546
1645,1200289,0.277546
1646,1272094,0.277546


In [ ]:
final_table = pd.merge(
    mentions_table,
    sentiment_table,
    on="player_ids",
    how="left"
).merge(
    topics_table,
    on="player_ids",
    how="left"
).merge(
    total_recency_table,
    on="player_ids",
    how="left"
)

In [ ]:
final_table

,player_ids,total_mentions,total_sentiment,avg_sentiment,total_transfer,total_injury,total_contract,total_performance,total_recency_score
0,3333,106,12.221728,0.115299,48,55,33,98,0.275923
1,3455,2,0.212278,0.106139,0,1,1,2,0.230630
2,4192,42,4.799540,0.114275,15,15,11,37,0.263786
3,5023,1,0.138602,0.138602,0,0,0,1,0.277546
4,7161,8,1.019362,0.127420,4,5,1,7,0.254314
...,...,...,...,...,...,...,...,...,...
1643,1084667,6,0.416138,0.069356,3,4,2,6,0.265712
1644,1176345,1,0.062885,0.062885,0,0,0,1,0.277546
1645,1200289,1,0.173349,0.173349,0,0,0,1,0.277546
1646,1272094,1,0.092112,0.092112,0,0,0,1,0.277546


In [ ]:
df.to_csv('player_popularity.csv', index=False)